读取文件

In [2]:
import pandas as pd
import numpy as np
import json

train_log = pd.read_csv("./data/train_logs.csv")
train_log.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


模拟写作过程

In [18]:
def simulate(row, content):
	activity = row.loc['activity']
	position = row.loc['cursor_position']
	down_event = row.loc['down_event']
	text_change = row.loc['text_change']
	
	if activity == 'Nonproduction':
		pass
	elif activity == 'Input':
		content = content[:position] + text_change + content[position:]
	elif activity == 'Remove/Cut':
		content = content[:position] + content[position + 1:] 
	elif activity == 'Paste':
		content = content[:position - len(text_change)] + text_change + content[position - len(text_change):]
	elif activity == 'Replace':
		replaces = text_change.split(' => ')
		position -= len(replaces[1])
		content = content[:position] + replaces[1] + content[position + len(replaces[0]):]
	elif 'Move From' in activity:
		activity = activity.replace('Move From ','')
		activity = activity.split(' To ')
		origin = json.loads(activity[0])
		target = json.loads(activity[1])
		content = content[:origin[0]] + content[origin[1]:]
		content = content[:target[0]] + text_change + content[target[0]:]

	return content

content = ''
first = train_log.loc[train_log['id'] == '001519c8']
for index, a in first.iterrows():
	content = simulate(row=a, content=content)

print(content)

'''
simulate(row=train_log.loc[2359], content="")
train_log.loc[train_log['activity'] == 'Paste'].head()
i = 2359
print(len(train_log.loc[i, 'text_change']))
print(train_log[i - 5 : i + 5])
print('hello' + train_log.loc[i,'text_change'] + 'world')
condition = ['Move From' in a for a in train_log['activity']]
train_log.loc[condition].head()
'''


qqqqqqqqqq qq qqqqq qq qqqq qqqq.  qqqqq qqq qqqq qqqqqq qq qq qqqqq qq qqqq qqqqq qq qqqqqqqqq qqqqq qqqq qqqqq qqqq qqqqqqqq qqqqqqqqq qqqq.  qqqqqq qqq qqqqq qqqq qqqqqqqqqq q,q qqq qqqqqqqqqq qqqqq qqq qqqqq qqqqqq qq qq qqqqq qqqqqqqqqq qqqqqqq qq qq qqqqqqqqqqq.  qqqqqqqq qq qqqqqqqqqqq qqqq qqq qqqqqqqqq qqq qqqqqqq qq qqqqqq qqqqq qqq qqq q qqqqqqqqq qq qq qqq qqqqq qqqqq
 qq qqq.
qq qq qqqq qqqq qqq qqqqqqqqq qqq qqqqqqq qq qqq qqqqq qqqqq, qq qq  qqqqqqqqq qqq qqqqqqqq qqqqq qq qqq qqqqqqqqqqq qq qqqqqqqqq.  qqqqqqqqq qq qqq qqqqqqqq qqq qqq qqqq qq qqqqqqq qqqqq qqqqq, qqq qqqqqq qqqqq qqqqq qqq qqq qq qqq qqqqqqq qqqqqqq qqqq.  qqqq qqqqq qqqqq qqqq qqqq 'qqqqqqq qqqqqqqqq qqqqq qqqqqqq qqqqqqq qqqqqqqqqq, qqqq qq qqqqqqqqqq qqqqqqq qqq qqqqqqq; qqqqqqq, qqqqq qqqqqqqq qqqqqq qqqqqqq qqqqqqq qqq qqqqq qqq qqq qqq qqqqqqq.  qqqq qqqqqqqqq qqqq qqq qqqq qqqq qq qqqqqqqqqqqqq qqqq qqqqq qqqqq.  qqq qqqqqqqqqq qq qqqqqqqq q qqqqqq, qqqqqqqq qqqq qqqq qqqqqqqqqq, qqq. qq qqqqq q

'\nsimulate(row=train_log.loc[2359], content="")\ntrain_log.loc[train_log[\'activity\'] == \'Paste\'].head()\ni = 2359\nprint(len(train_log.loc[i, \'text_change\']))\nprint(train_log[i - 5 : i + 5])\nprint(\'hello\' + train_log.loc[i,\'text_change\'] + \'world\')\ncondition = [\'Move From\' in a for a in train_log[\'activity\']]\ntrain_log.loc[condition].head()\n'